### Data Preprocessing

3. For each stock in the dataset you should execute a rolling regression of log-returns as prescribed by the Market Model.

4. The sample for the rolling regression must be 180 days. You will lose the first 180 days of the sample.

Imports

In [1]:
import numpy as np
import pandas as pd

import time
import datetime as dt # for defining dates

import matplotlib.pyplot as plt # plotting
import matplotlib.dates as mdates # styling dates
%matplotlib inline

from functions import *

In [2]:
from rich.progress import track
import os
import math
import statistics as st

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import regression

Tickers

In [ ]:
'''
tickers_ES50 = [line.strip() for line in open("../data/tickers_ES50.txt", 'r')]
tickers_ES50 = ['^STOXX50E'] + tickers_ES50
tickers_ES50
'''

In [ ]:
'''
ticker_ndx = ['^STOXX50E']
ticker_ndx
'''

Costants

In [4]:
# folder used to store stock data
folder = "../data/"

# size of the window used for rolling regression
sample_size = 180

Read Data

In [5]:
df_ES50 = pd.read_csv('../data/raw/ES50.csv', header=1)
df_ES50 = df_ES50.iloc[:, : 52]
df_ES50 = df_ES50.drop([0])
df_ES50 = df_ES50.rename(columns={"Symbols": "Date", "^STOXX50E": "EuroStoxx50"})
df_ES50['Date'] = pd.to_datetime(df_ES50['Date'])
df_ES50.head()

,Date,EuroStoxx50,ADS.DE,ADYEN.AS,AD.AS,AI.PA,AIR.PA,ALV.DE,ABI.BR,ASML.AS,...,SAF.PA,SAN.PA,SAP.DE,SU.PA,SIE.DE,STLA.MI,TTE.PA,DG.PA,VOW.DE,VNA.DE
1,2016-01-04,3164.760010,78.004707,NaN,15.247333,63.063118,53.968327,111.405846,90.266624,74.629242,...,57.882877,59.244190,63.260578,41.909256,68.110123,4.946972,27.490080,47.195293,117.650673,22.053017
2,2016-01-05,3178.010010,77.737396,NaN,15.321729,62.687923,55.059048,111.762909,91.510849,75.216354,...,57.576126,59.946838,64.022430,42.097519,68.540154,5.077475,27.239086,47.456890,111.036285,22.200811
3,2016-01-06,3139.320068,76.837410,NaN,15.223837,61.812473,54.331902,112.119972,90.306778,73.464317,...,57.167126,59.137260,64.578133,41.352646,68.269386,4.813435,26.977922,47.178940,110.737564,21.741453
4,2016-01-07,3084.679932,75.643364,NaN,15.145526,60.386761,53.241184,109.799019,88.781578,71.749557,...,56.051670,58.495708,63.941761,40.501366,66.963402,4.655616,26.323318,46.639381,105.446068,21.334026
5,2016-01-08,3033.469971,74.360207,NaN,15.176851,59.467541,52.106125,108.406448,86.333267,68.720772,...,55.410286,57.029305,63.681831,40.644611,66.501534,4.464415,25.336323,46.484043,106.726273,20.994507


In [6]:
# copy for later
f = df_ES50

Calculate Log Returns

In [9]:
# calculate log-returns
ES50_returns = get_log_returns(f)

# manage null values
ES50_returns.fillna(np.nan, inplace=True)

# extract only the rows where ndx is not NaN
ES50_returns = ES50_returns[ES50_returns['EuroStoxx50'].notna()]

# save to CSV
ES50_returns.to_csv(folder + "ES50_log-ret.csv")

ES50_returns.head()

,Date,EuroStoxx50,ADS.DE,ADYEN.AS,AD.AS,AI.PA,AIR.PA,ALV.DE,ABI.BR,ASML.AS,...,SAF.PA,SAN.PA,SAP.DE,SU.PA,SIE.DE,STLA.MI,TTE.PA,DG.PA,VOW.DE,VNA.DE
2,2016-01-05,0.004178,-0.003433,NaN,0.004867,-0.005967,0.020009,0.003200,0.013690,0.007836,...,-0.005314,0.011790,0.011971,0.004482,0.006294,0.026038,-0.009172,0.005528,-0.057863,0.006679
3,2016-01-06,-0.012249,-0.011645,NaN,-0.006410,-0.014064,-0.013295,0.003190,-0.013245,-0.023569,...,-0.007129,-0.013597,0.008642,-0.017852,-0.003958,-0.053403,-0.009634,-0.005874,-0.002694,-0.020908
4,2016-01-07,-0.017558,-0.015662,NaN,-0.005157,-0.023335,-0.020279,-0.020918,-0.017033,-0.023618,...,-0.019705,-0.010908,-0.009903,-0.020801,-0.019315,-0.033337,-0.024564,-0.011502,-0.048963,-0.018917
5,2016-01-08,-0.016741,-0.017109,NaN,0.002066,-0.015339,-0.021550,-0.012764,-0.027964,-0.043130,...,-0.011509,-0.025388,-0.004073,0.003531,-0.006921,-0.041936,-0.038216,-0.003336,0.012068,-0.016042
6,2016-01-11,-0.001973,0.021458,NaN,0.008222,-0.012060,-0.011123,0.003945,0.019794,0.026101,...,-0.016918,-0.013754,0.007711,-0.019317,-0.006487,-0.004087,0.000000,0.021402,0.013503,0.003987


Sample

In [10]:
def get_sample_returns(days):
    sample_returns = ES50_returns.iloc[days: days + sample_size]
    sample_returns = sample_returns.reset_index().drop(['index'], axis=1)

    # remove columns containing null values (drop the entire column if at least one NaN)
    sample_returns.dropna(axis=1, how='any', inplace=True)

    # extract the name of the columns
    tickers = sample_returns.columns.tolist()

    # remove the first two element in indexNames (Date, EuroStoxx50) because I don't need them
    tickers = tickers[2:]

    return sample_returns, tickers

In [11]:
# rolling_df contains the first 180 rows
rolling_df, tickers = get_sample_returns(0)
print("{} days, {} stocks".format(sample_size, len(rolling_df.columns)))
rolling_df.head()

180 days, 43 stocks


,Date,EuroStoxx50,ADS.DE,AD.AS,AI.PA,AIR.PA,ALV.DE,ABI.BR,ASML.AS,CS.PA,...,PHIA.AS,SAF.PA,SAN.PA,SAP.DE,SU.PA,SIE.DE,TTE.PA,DG.PA,VOW.DE,VNA.DE
0,2016-01-05,0.004178,-0.003433,0.004867,-0.005967,0.020009,0.003200,0.013690,0.007836,0.000411,...,0.002398,-0.005314,0.011790,0.011971,0.004482,0.006294,-0.009172,0.005528,-0.057863,0.006679
1,2016-01-06,-0.012249,-0.011645,-0.006410,-0.014064,-0.013295,0.003190,-0.013245,-0.023569,-0.003704,...,-0.012267,-0.007129,-0.013597,0.008642,-0.017852,-0.003958,-0.009634,-0.005874,-0.002694,-0.020908
2,2016-01-07,-0.017558,-0.015662,-0.005157,-0.023335,-0.020279,-0.020918,-0.017033,-0.023618,-0.023570,...,-0.009078,-0.019705,-0.010908,-0.009903,-0.020801,-0.019315,-0.024564,-0.011502,-0.048963,-0.018917
3,2016-01-08,-0.016741,-0.017109,0.002066,-0.015339,-0.021550,-0.012764,-0.027964,-0.043130,-0.018963,...,-0.015014,-0.011509,-0.025388,-0.004073,0.003531,-0.006921,-0.038216,-0.003336,0.012068,-0.016042
4,2016-01-11,-0.001973,0.021458,0.008222,-0.012060,-0.011123,0.003945,0.019794,0.026101,0.005149,...,0.016348,-0.016918,-0.013754,0.007711,-0.019317,-0.006487,0.000000,0.021402,0.013503,0.003987


In [12]:
print(rolling_df.shape)
print(rolling_df.columns)

(180, 43)
Index(['Date', 'EuroStoxx50', 'ADS.DE', 'AD.AS', 'AI.PA', 'AIR.PA', 'ALV.DE',
       'ABI.BR', 'ASML.AS', 'CS.PA', 'BAS.DE', 'BAYN.DE', 'BBVA.MC', 'SAN.MC',
       'BMW.DE', 'BNP.PA', 'BN.PA', 'DB1.DE', 'DPW.DE', 'DTE.DE', 'EL.PA',
       'RMS.PA', 'IBE.MC', 'ITX.MC', 'IFX.DE', 'INGA.AS', 'KER.PA', 'OR.PA',
       'LIN.DE', 'MC.PA', 'MBG.DE', 'MUV2.DE', 'RI.PA', 'PHIA.AS', 'SAF.PA',
       'SAN.PA', 'SAP.DE', 'SU.PA', 'SIE.DE', 'TTE.PA', 'DG.PA', 'VOW.DE',
       'VNA.DE'],
      dtype='object')


Rolling Regression

$r_i − r_f = \alpha_i + \beta_i(R_M − r_f) + e_i$

For each stock:

- $r_i$: log-returns for stock i (ok)
- $r_f$: risk-free rate (not considered)
- $\alpha_i$: excess returns (intercept of the regression)
- $\beta_i$: coefficient of the regression
- $R_M$: log-returns of the market (ok)
- $e_i$: specific risk of the stock

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
# columns
rank_df = pd.DataFrame(columns=['tickers', 'r2', 'beta', 'alpha', 'alpha_significance', 'absolute_returns', 'specific_risk', 'systematic_risk'])

ndx_returns = rolling_df.iloc[:, 1].values

for ticker in tickers:
    ticker_returns = rolling_df.iloc[0 : sample_size, rolling_df.columns.get_loc(ticker)]

    ndx_returns = sm.add_constant(ndx_returns)
    model = sm.OLS(ticker_returns, ndx_returns)
    result = model.fit()

    rank_df = rank_df.append({'tickers': ticker, 'r2': result.rsquared, 'beta': result.params[1], 'alpha': result.params[0], 'alpha_significance': result.pvalues[0], 'absolute_returns': np.sum(ticker_returns),  'specific_risk': result.resid.std(), 'systematic_risk': result.params[1] ** 2 * ndx_returns.std() ** 2}, ignore_index=True)

In [16]:
rank_df['total_risk'] = rank_df['specific_risk'] + rank_df['systematic_risk']

In [17]:
rank_df.head()

,tickers,r2,beta,alpha,alpha_significance,absolute_returns,specific_risk,systematic_risk,total_risk
0,ADS.DE,0.177656,0.41288,0.003112,0.002831,0.559717,0.013751,0.042638,0.056389
1,AD.AS,0.319903,0.473584,0.000715,0.371868,0.128262,0.010689,0.056098,0.066787
2,AI.PA,0.536992,0.719739,0.000329,0.671121,0.058461,0.010346,0.129569,0.139915
3,AIR.PA,0.587508,0.947977,-0.000391,0.671284,-0.0713,0.012296,0.224775,0.237071
4,ALV.DE,0.81318,1.058462,-0.000226,0.700797,-0.041755,0.007854,0.280222,0.288076


In [18]:
print(rank_df.shape)
print(rank_df.columns)

(41, 9)
Index(['tickers', 'r2', 'beta', 'alpha', 'alpha_significance',
       'absolute_returns', 'specific_risk', 'systematic_risk', 'total_risk'],
      dtype='object')


In [19]:
rank_df.to_csv('../data/ES50_parameters.csv')